# RedPanda Examples

In [1]:
import pandas, redpanda, sqlalchemy, sys
print("RedPanda version:", redpanda.__version__)
print("Pandas version:", pandas.__version__)
print("Python version:", sys.version)

RedPanda version: 0.3.1
Pandas version: 0.20.3
Python version: 3.6.1 (default, May  2 2017, 15:16:41) 
[GCC 6.3.0]


## Opening a session

Open a `RedPanda` session to begin exploring data:

In [2]:
engine = redpanda.create_engine("sqlite://")
Session = redpanda.orm.sessionmaker(bind=engine)
session = Session()

Note that this is syntactic sugar for SQLAlchemy calls:

In [3]:
engine = sqlalchemy.create_engine("sqlite://")
sessionmaker = sqlalchemy.orm.sessionmaker(class_=redpanda.orm.Session,
                                           query_cls=redpanda.orm.Query)
session = sessionmaker(bind=engine)

## Creating An Example Model

Define your models using the declarative API. `RedPanda` defines one such model in the `example` module:

In [4]:
import sqlalchemy.ext.declarative
Base = sqlalchemy.ext.declarative.declarative_base()
class Widget(Base):
    id        = sqlalchemy.Column(sqlalchemy.Integer, primary_key=True)
    timestamp = sqlalchemy.Column(sqlalchemy.DateTime)
    name      = sqlalchemy.Column(sqlalchemy.String)
    kind      = sqlalchemy.Column(sqlalchemy.String)
    units     = sqlalchemy.Column(sqlalchemy.Integer)

    __read_sql__  = {"index_col": ["timestamp"], "parse_dates": ["timestamp"]}
    __tablename__ = "widgets"

Defining the class attribute `__read_sql__` will configure default arguments passed to `pandas.read_sql()`. 

In this case we are configuring the `widgets` table to use the `timestamp` column as an index and that we wish for `pandas` to parse this column as a `datetime` object.

In [5]:
from redpanda.example import create_widgets
create_widgets(session)

## Querying Data

Use the `frame()` method of a query to return the results as a `DataFrame`:

In [6]:
session.query(Widget).frame().head()

,id,name,kind,units
timestamp,,,,
2016-01-01 13:05:00,1,compromises,bopper,64
2016-01-02 21:23:00,2,tap,fizzer,82
2016-01-15 23:21:00,3,points,buzzer,62
2016-01-24 01:35:00,4,relocations,bopper,70
2016-01-27 04:17:00,5,keel,fizzer,90


### Querying with Filters

Use SQLAlchemy filters to refine the query:

In [7]:
session.query(Widget).filter(Widget.timestamp.between("2016-11-01", "2016-11-30")).frame()

,id,name,kind,units
timestamp,,,,
2016-11-02 09:07:00,61,artilleries,buzzer,44
2016-11-08 16:32:00,62,users,fizzer,36
2016-11-09 10:08:00,63,carloads,buzzer,29
2016-11-10 07:41:00,64,vine,buzzer,54
2016-11-11 07:33:00,65,crosses,fizzer,4
2016-11-12 19:53:00,66,concerns,buzzer,90
2016-11-14 02:15:00,67,gum,bopper,44
2016-11-22 17:07:00,68,instruments,bopper,99
2016-11-28 20:44:00,69,rescuers,buzzer,34


Alternatively, use the `within()` method to query an instrumented attribute with a pandas index:

In [8]:
index = pandas.period_range("2016-11-01", "2016-11-30", freq="W")
session.query(Widget).filter(Widget.timestamp.within(index)).frame()

,id,name,kind,units
timestamp,,,,
2016-10-31 19:30:00,60,acceptor,fizzer,39
2016-11-02 09:07:00,61,artilleries,buzzer,44
2016-11-08 16:32:00,62,users,fizzer,36
2016-11-09 10:08:00,63,carloads,buzzer,29
2016-11-10 07:41:00,64,vine,buzzer,54
2016-11-11 07:33:00,65,crosses,fizzer,4
2016-11-12 19:53:00,66,concerns,buzzer,90
2016-11-14 02:15:00,67,gum,bopper,44
2016-11-22 17:07:00,68,instruments,bopper,99


### Overriding Defaults

Pass arguments to `pandas.read_sql()` into the `frame()` method:

In [9]:
session.query(Widget).frame(index_col="id").head()

,timestamp,name,kind,units
id,,,,
1,2016-01-01 13:05:00,compromises,bopper,64
2,2016-01-02 21:23:00,tap,fizzer,82
3,2016-01-15 23:21:00,points,buzzer,62
4,2016-01-24 01:35:00,relocations,bopper,70
5,2016-01-27 04:17:00,keel,fizzer,90


## Writing DataFrame to Session

Use the `add_dataframe()` method of the `RedPanda` session to place `DataFrame` rows into the session as models:

In [10]:
from datetime import datetime
dfdata = {
    "kind": "newkind", 
    "name": "newname", 
    "units": 1000, 
    "timestamp": datetime.utcnow()}
addme = pandas.DataFrame(dfdata, index=[0])
addme

,kind,name,timestamp,units
0,newkind,newname,2017-08-31 15:38:27.635330,1000


In [11]:
session.add_dataframe(Widget, addme)
session.commit()
session.query(Widget).filter(Widget.kind=="newkind").frame()

,id,name,kind,units
timestamp,,,,
2017-08-31 15:38:27.635330,76,newname,newkind,1000
